In [1]:
# import library
import tensorflow as tf
import numpy as np
import sys, os,cv2
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from skimage.transform import resize
from scipy.misc import imread
from imgaug import augmenters as iaa
import nibabel as nib
import imgaug as ia
from scipy.ndimage import zoom
from sklearn.utils import shuffle
from skimage import util 
import matplotlib.animation as animation

plt.style.use('seaborn-white')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
np.random.seed(6278)
tf.set_random_seed(6728)
ia.seed(6278)

# Generate training data
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# get the data as well as show image function
from ipynb.fs.full.z_util  import show_images,get_train,get_test,get_unlabel
from sklearn.preprocessing import OneHotEncoder
train_data,train_label = get_train()
test_data ,test_label  = get_test()
unlabeled              = get_unlabel()

In [87]:
# one hot encode the labels
np.set_printoptions(threshold=np.nan)
encoder = OneHotEncoder(sparse =True)
train_label_encoded = encoder.fit_transform(train_label[:,None]).toarray()
test__label_encoded = encoder.fit_transform(test_label [:,None]).toarray()
print(train_label[:3],'\n',train_label_encoded[:3])
print(test_label[:3], '\n',test__label_encoded[:3])

train_data = (train_data - train_data.min((1,2),keepdims=True))/(train_data.max((1,2),keepdims=True)-train_data.min((1,2),keepdims=True))
test_data  = (test_data -  test_data.min((1,2),keepdims=True))/( test_data.max((1,2),keepdims=True)- test_data.min((1,2),keepdims=True))

print(train_data.mean(),test_data.mean())

[2 6 2] 
 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[7 8 6] 
 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
0.43950329205254085 0.4383057132586417


In [ ]:
# show some images of train test and unlabeld
show_images(train_data,train_label)
show_images(test_data,test_label)
show_images(unlabeled,None)

In [81]:
# import layers
def tf_relu(x):    return tf.nn.relu(x)
def d_tf_relu(x):  return tf.cast(tf.greater(x,0),tf.float64)

def tf_elu(x):     return tf.nn.elu(x)
def d_tf_elu(x):   return tf.cast(tf.greater(x,0),tf.float64)  + (tf_elu(tf.cast(tf.less_equal(x,0),tf.float64) * x) + 1.0)

def tf_softmax(x): return tf.nn.softmax(x)

class CNN():

    def __init__(self,k,inc,out, stddev=0.05,which_reg=0,act=tf_elu,d_act=d_tf_elu):
        self.w          = tf.Variable(tf.random_normal([k,k,inc,out],stddev=stddev,seed=2,dtype=tf.float64))
        self.b          = tf.Variable(tf.zeros(out,dtype=tf.float64))
        self.m,self.v   = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.mb,self.vb = tf.Variable(tf.zeros_like(self.b)),tf.Variable(tf.zeros_like(self.b))
        self.act,self.d_act = act,d_act
        self.which_reg  = which_reg
        
    def getw(self): return [self.w,self.b]

    def feedforward(self,input,stride=1,padding='VALID'):
        self.input  = input
        self.layer  = tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding) + self.b 
        self.layerA = self.act(self.layer)
        return self.layerA

    def backprop(self,gradient,stride=1,padding='VALID'):
        grad_part_1 = gradient
        grad_part_2 = self.d_act(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2
        grad_b      = tf.reduce_mean(grad_middle,(0,1,2))/batch_size
        grad        = tf.nn.conv2d_backprop_filter(input = grad_part_3,filter_sizes = tf.shape(self.w),  out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding) / batch_size
        grad_pass   = tf.nn.conv2d_backprop_input (input_sizes = tf.shape(self.input),filter= self.w,out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding)

        if self.which_reg == 0:   grad = grad
        if self.which_reg == 0.5: grad = grad + lamda * (tf.sqrt(tf.abs(self.w))) * (1.0/tf.sqrt(tf.abs(self.w)+ 10e-5)) * tf.sign(self.w)
        if self.which_reg == 1:   grad = grad + lamda * tf.sign(self.w)
        if self.which_reg == 1.5: grad = grad + lamda * 1.0/(tf.sqrt(tf.square(self.w) + 10e-5)) * self.w
        if self.which_reg == 2:   grad = grad + lamda * (1.0/tf.sqrt(tf.square(tf.abs(self.w))+ 10e-5)) * tf.abs(self.w) * tf.sign(self.w)
        if self.which_reg == 2.5: grad = grad + lamda * 2.0 * self.w
        if self.which_reg == 3:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),3)+ 10e-5,-0.66) * tf.pow(tf.abs(self.w),2) * tf.sign(self.w)
        if self.which_reg == 4:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),4)+ 10e-5,-0.75) * tf.pow(tf.abs(self.w),3) * tf.sign(self.w)

        update_w = []
        
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1) ; v_hat = self.v / (1-beta2)
        adam_middle = m_hat * learning_rate/(tf.sqrt(v_hat) + adam_e)
        update_w.append(tf.assign(self.w,tf.subtract(self.w,adam_middle  )))
        
        update_w.append(tf.assign( self.mb,self.mb*beta1 + (1-beta1) * (grad_b)   ))
        update_w.append(tf.assign( self.vb,self.vb*beta2 + (1-beta2) * (grad_b ** 2)   ))
        m_hatb = self.mb / (1-beta1) ; v_hatb = self.vb / (1-beta2)
        adam_middleb = m_hatb * learning_rate/(tf.sqrt(v_hatb) + adam_e)
        update_w.append(tf.assign(self.b,tf.subtract(self.b,adam_middleb  )))
        
        return grad_pass,update_w

In [82]:
# hyper parameters and layers
num_epoch = 100; learning_rate = 0.0008; batch_size = 20 ; print_size= 10
beta1,beta2,adam_e = 0.9,0.999,1e-8

l1 = CNN(3,3, 32)
l2 = CNN(3,32,32)
l3 = CNN(3,32,32)

l4 = CNN(3,32,64)
l5 = CNN(3,64,64)
l6 = CNN(3,64,10)

In [84]:
# create graph
x = tf.placeholder(shape=[batch_size,96,96,3],dtype=tf.float64)
y = tf.placeholder(shape=[batch_size,10],dtype=tf.float64)
temperature = tf.cast(tf.placeholder_with_default(1.0,shape=[]),tf.float64)

layer1      = l1.feedforward(x)
layer2      = l2.feedforward(layer1)
layer3      = l3.feedforward(layer2)
layer3_pool = tf.nn.avg_pool(layer3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')

layer4      = l4.feedforward(layer3_pool)
layer5      = l5.feedforward(layer4)
layer6      = l6.feedforward(layer5)

final_squeeze_temp = tf.squeeze(tf.reduce_mean(layer6,(1,2)))/temperature
final_prediction   = tf_softmax(final_squeeze_temp) 
cost               = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=final_squeeze_temp,labels=y))
accuracy           = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, -1), tf.argmax(final_prediction, -1)),tf.float64))

grad        = (final_prediction-y)[:,None,None,:]/batch_size
grad_expand = tf.tile(grad,[1,39,39,1])/temperature
grad6,grad6_up = l6.backprop(grad_expand)
grad5,grad5_up = l5.backprop(grad6)
grad4,grad4_up = l4.backprop(grad5)

grad3_uppool   = tf.tile(grad4,[1,2,2,1])
grad3,grad3_up = l3.backprop(grad3_uppool)
grad2,grad2_up = l2.backprop(grad3)
grad1,grad1_up = l1.backprop(grad2)

grad_update    = grad6_up + grad5_up + grad4_up + grad3_up + grad2_up + grad1_up

In [86]:
# start the session
# sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for iter in range(num_epoch):
    for current_data_index in range(0,len(train_data),batch_size):
        current_data  = train_data[current_data_index:current_data_index+batch_size]
        current_label = train_label_encoded[current_data_index:current_data_index+batch_size]  
        sess_results  = sess.run([accuracy,cost,grad_update],feed_dict={x:current_data,y:current_label,temperature:1.5})
        sys.stdout.write('\riter: ' + str(iter) + " index: " + str(current_data_index) + '/' + str(len(train_data)) + 
                         " acc: " + str(sess_results[0]) + ' cost: ' + str(np.around(sess_results[1],3))); sys.stdout.flush()
    if iter % print_size == 0 : print('\n-------------')

iter: 0 index: 380/5000 acc: 0.1 cost: 2.4941

KeyboardInterrupt: 

In [44]:
# mid training and testing section
mid_test_data    = train_data[:batch_size,:,:,:]; mid_test_label = train_label_encoded[:batch_size,:]
mid_sess_results = sess.run(final_prediction,feed_dict={x:mid_test_data,temperature:1.5}) 
print(np.around(mid_sess_results,3))
print(mid_test_label)
print('-------------------')
print(np.argmax(np.around(mid_sess_results,3),1))
print(np.argmax(mid_test_label  ,-1))

[[0.    0.007 0.    0.012 0.953 0.012 0.008 0.008 0.    0.   ]
 [0.    0.002 0.    0.003 0.957 0.008 0.023 0.006 0.    0.   ]
 [0.    0.008 0.    0.029 0.826 0.056 0.04  0.039 0.    0.001]
 [0.    0.004 0.001 0.013 0.261 0.051 0.587 0.059 0.    0.023]
 [0.    0.003 0.    0.005 0.979 0.004 0.007 0.002 0.    0.   ]]
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
-------------------
[4 4 4 6 4]
[1 5 1 6 3]


# Reference
1. file, i. (2018). import a function from another .ipynb file. Stack Overflow. Retrieved 11 November 2018, from https://stackoverflow.com/questions/44116194/import-a-function-from-another-ipynb-file
2. array?, H. (2018). How to print the full NumPy array?. Stack Overflow. Retrieved 11 November 2018, from https://stackoverflow.com/questions/1987694/how-to-print-the-full-numpy-array
3. Label Encoder vs. One Hot Encoder in Machine Learning. (2018). Medium. Retrieved 11 November 2018, from https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621
4. API Reference — scikit-learn 0.20.0 documentation. (2018). Scikit-learn.org. Retrieved 11 November 2018, from https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing
5. Why is there no support for directly computing cross entropy? · Issue #2462 · tensorflow/tensorflow. (2018). GitHub. Retrieved 11 November 2018, from https://github.com/tensorflow/tensorflow/issues/2462
6. argmax?, E. (2018). Evaluating softmax (classification) accuracy in Tensorflow with round or argmax?. Stack Overflow. Retrieved 11 November 2018, from https://stackoverflow.com/questions/41759771/evaluating-softmax-classification-accuracy-in-tensorflow-with-round-or-argmax
7. tensorflow.placeholder_with_default Python Example. (2018). Programcreek.com. Retrieved 11 November 2018, from https://www.programcreek.com/python/example/90324/tensorflow.placeholder_with_default
8. JaeDukSeo/Daily-Neural-Network-Practice-2. (2018). GitHub. Retrieved 11 November 2018, from https://github.com/JaeDukSeo/Daily-Neural-Network-Practice-2/blob/master/NeuralNetwork/ELU/a%20paper%20adam.py
9. 